In [1]:
import torch
import numpy as np
from deeprobust.graph.data import Dataset
from deeprobust.graph.defense import GCN as GCN_DR
from deeprobust.graph.targeted_attack import FGA

In [2]:
from greedy_mcmc_attack import *
from collections import defaultdict
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [3]:
cora_dataset = Planetoid(root='', name='Citeseer')
data = cora_dataset[0].to(device)
print(data)
idx_train, idx_val, idx_test = data.train_mask.nonzero(as_tuple=True)[0].tolist(), data.val_mask.nonzero(as_tuple=True)[0].tolist(), data.test_mask.nonzero(as_tuple=True)[0].tolist()

Data(x=[3327, 3703], edge_index=[2, 9104], y=[3327], train_mask=[3327], val_mask=[3327], test_mask=[3327])


In [4]:
data_dr = Dataset(root='/tmp/', name='citeseer', setting='nettack')
adj, features, labels = data_dr.adj, data_dr.features, data_dr.labels
# idx_train, idx_val, idx_test = data_dr.idx_train, data_dr.idx_val, data_dr.idx_test
idx_unlabeled = np.union1d(idx_val, idx_test)

Loading citeseer dataset...
Selecting 1 largest connected components


In [5]:
degrees = np.array(adj.sum(axis=1)).flatten()
max_degree_vertex = degrees.argmax()

In [6]:
surrogate = GCN_DR(nfeat=features.shape[1], nclass=labels.max().item()+1, nhid=16,
                with_relu=False, device=device)
surrogate = surrogate.to(device)
surrogate.fit(features, adj, labels, idx_train)

In [7]:
ptb_rate = 0.1

In [8]:
model = FGA(surrogate, nnodes=adj.shape[0], attack_structure=True, attack_features=False, device=device)
model = model.to(device)
perturbations = int(ptb_rate * (adj.sum() // 2))
model.attack(features, adj, labels, idx_train, max_degree_vertex, n_perturbations=perturbations)
modified_adj = model.modified_adj

### arch tests

In [18]:
model = GCN(data.x.shape[1], cora_dataset.num_classes, [16]).to(device)

In [19]:
model_save_path = "../models/citeseer_gcn_model.pth"
list_save_path = "../attacks/citeseer_gcn_edges.pth"

In [20]:
model, edges_to_add, train = load_model_and_edges(model_save_path, list_save_path, model, device)

In [21]:
initial_loss, initial_accuracy = train.test(data)
print(f"Initial Accuracy: {initial_accuracy}")
print(f"Initial Loss: {initial_loss}")

Initial Accuracy: 0.69
Initial Loss: 1.2167096138000488


In [22]:
from scipy.sparse import csr_matrix

adj1 = data_dr.adj.tocsr()
adj2 = modified_adj.tocsr()

arr_diff = (adj1 - adj2).tocoo()

In [23]:
arr_diff

<2110x2110 sparse matrix of type '<class 'numpy.float32'>'
	with 732 stored elements in COOrdinate format>

In [24]:
data_modded = copy.deepcopy(data)

In [25]:
# thanks gpt
existing = data_modded.edge_index.t().cpu().numpy()

diff_coo = arr_diff.tocoo()
diff_edges = np.stack([diff_coo.row, diff_coo.col], axis=1)

existing_set = set(map(tuple, existing))
diff_set = set(map(tuple, diff_edges))
diff_set_full = diff_set | set((v, u) for u, v in diff_set)

cleaned_edges = existing_set - diff_set_full
print(len(cleaned_edges))

added_edges = diff_set_full - existing_set
print(len(added_edges))

final_edges = cleaned_edges | added_edges

final_edges = torch.tensor(list(final_edges), dtype=torch.long).t().to(device)
data_modded.edge_index = final_edges

9104
732


In [26]:
new_loss, new_accuracy = train.test(data_modded)

print(f"Evasion Change in Accuracy: {(100 * ((new_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Evasion Change in Loss: {(100 * ((new_loss - initial_loss) / initial_loss)):.4f}%")

### poisoning
model_poisoning = GCN(data_modded.x.shape[1], cora_dataset.num_classes, [16]).to(device)
model_poisoning.reset_parameters()
train_modded = Trainable(model_poisoning)
train_modded.fit(data_modded, 1000)

poisoning_loss, poisoning_accuracy = train_modded.test(data_modded)
print(f"Poisoning Change in Accuracy: {(100 * ((poisoning_accuracy - initial_accuracy) / initial_accuracy)):.4f}%")
print(f"Poisoning Change in Loss: {(100 * ((poisoning_loss - initial_loss) / initial_loss)):.4f}%")

Evasion Change in Accuracy: -0.1449%
Evasion Change in Loss: -1.7357%
Epoch 0, Train Loss - 1.7856364250183105, Val Loss - 1.72316312789917, Val Accuracy - 0.506
Epoch 20, Train Loss - 0.11799069494009018, Val Loss - 1.0272996425628662, Val Accuracy - 0.666
Epoch 40, Train Loss - 0.05477059260010719, Val Loss - 1.096648931503296, Val Accuracy - 0.68
Epoch 60, Train Loss - 0.04265357181429863, Val Loss - 1.0711075067520142, Val Accuracy - 0.678
Epoch 80, Train Loss - 0.07732152193784714, Val Loss - 1.0922363996505737, Val Accuracy - 0.678
Epoch 100, Train Loss - 0.07432007044553757, Val Loss - 1.1023225784301758, Val Accuracy - 0.686
Epoch 120, Train Loss - 0.03718380630016327, Val Loss - 1.133830189704895, Val Accuracy - 0.68
Epoch 140, Train Loss - 0.054592110216617584, Val Loss - 1.1571366786956787, Val Accuracy - 0.684
Epoch 160, Train Loss - 0.03617800027132034, Val Loss - 1.1248778104782104, Val Accuracy - 0.684
Epoch 180, Train Loss - 0.03651595488190651, Val Loss - 1.20340335369